In [1]:
import os

import simpleneighbors
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
import tensorflow_text

import _pickle as pickle
from bs4 import BeautifulSoup
import re
import pymorphy2
from IPython.display import clear_output

In [2]:
"""
Парсинг практически совпадает с тем, что было в ноутбуке с BM25F
"""

def url_doc():
    with open ('urls.numerate.txt') as f:
        line = True
        u_d = {}
        while line:
            line = f.readline()
            if line:
                id_url = line.split()
                u_d[id_url[1]] = id_url[0]
    return u_d

In [3]:
def open_files():
    folders = ['./content/content/20170702/', './content/content/20170704/', './content/content/20170707/',\
               './content/content/20170708/', './content/content/20170710/', './content/content/20170711/',\
               './content/content/20170717/', './content/content/20170726/']
    i = 0
    for cur_folder in folders:
        for file in os.listdir(cur_folder):
            parser(cur_folder+file)
            clear_output(wait=True)
            print('Loaded {} files, {} to go'.format(i, 38114 - i))
            i+=1

In [4]:
def parser(way):
    
    with open(way, encoding='utf-8', errors='ignore') as file:
        url = file.readline()[:-1]
        
        raw_html = file.read()
        soup = BeautifulSoup(raw_html, 'html')
        
        text = ''
        text_lemma = ''
        
        for obj in soup.find_all(['title']):
            
            text = obj.text
            
            words_list = text.split()
            
            for string in words_list:
                text_lemma += string + ' '
                
            text_lemma = text_lemma[:-1]
        
        tuple_ = (text_lemma, len(text_lemma))  
                    
    save_tuple(tuple_, URL_DOC[url])

In [5]:
def save_tuple(tuple_, docid):
    with open('./TuplesForNN/'+docid+'.pkl', 'wb') as out:
        pickle.dump(tuple_, out, 2)

In [6]:
def open_tuple(docid):
    with open('./TuplesForNN/'+docid+'.pkl', 'rb') as inp:
        tuple_ = pickle.load(inp)
    return tuple_

In [7]:
def requests():
    dict_ = {}
    text = ''
    with open('Q_NUMERATE.txt', encoding='utf8') as f:
        line = f.readline()
        while line:
            list_ = line.split()
            for word in list_[1:]:
                text += word + ' '
            text = text[:-1]
            
            dict_[list_[0]] = text
    
            line = f.readline()
            text = '' 
    
    return dict_

In [8]:
def qnum_docs():
    dict_ = {}
    with open('sample.technosphere.ir1.textrelevance.submission.txt') as f:
        line = f.readline()
        while line:
            line = f.readline()
            if line:
                line = line[:-1].split(',')
                if line[0] in dict_:
                    dict_[line[0]] += [line[1]]
                else:
                    dict_[line[0]] = [line[1]]
    return dict_

In [9]:
def title_docid():
    t_d = {}
    for qnum in QNUM_D:
        
        for docid in QNUM_D[qnum]:
            title, len_ = open_tuple(docid)
            t_d[title[:150]] = docid
        clear_output(wait=True)
        print('{} querries handled'.format(qnum))
    return t_d

In [10]:
def get_index(qnum):
    
    list_docs = QNUM_D[qnum]
    title_list = []
    
    for doc in list_docs:
        title_len = open_tuple(doc)

        text = title_len[0]
        text = re.sub('\W+', ' ', text.lower())
        
        if title_len[0] != '':
            title_list += [text[:150]]

    encodings = model.signatures['response_encoder'](input=tf.constant(title_list),
                                                     context=tf.constant(title_list))

    index = simpleneighbors.SimpleNeighbors(len(encodings['outputs'][0]), metric='angular')

    for ind, title in enumerate(title_list):
        index.add_one(title, encodings['outputs'][ind])

    index.build()
            
    return index

In [11]:
def FinalRound(_continue=1):
    
    for qnum in QNUM_Q:
        if int(qnum) >= _continue:
            index = get_index(qnum)
            query_embedding = model.signatures['question_encoder']\
                                (tf.constant([QNUM_Q[qnum]]))['outputs'][0]
            nearest_10 = index.nearest(query_embedding, n=10)
            with open('./submissions/NN_3.txt', 'a') as f:
                if qnum =='1':
                    f.write('QueryId,DocumentId\n')

                for title in nearest_10:
                    f.write(str(qnum)+','+str(TITLE_DOC[title])+'\n')

        clear_output(wait=True)
        print('{} queries handled'.format(qnum))

In [12]:
def on_start():
    def printer(r):
        clear_output(wait=True)
        print('URL_DOC____{}'.format(r[0]))
        print('model______{}'.format(r[1]))
        print('QNUM_Q_____{}'.format(r[2]))
        print('QNUM_D_____{}'.format(r[3]))
        print('TITLE_DOC__{}'.format(r[4]))
        
    global URL_DOC, model, QNUM_Q, QNUM_D, TITLE_DOC
    printer(['X','X','X','X','X'])
    URL_DOC = url_doc()
    printer(['V','X','X','X','X'])
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3")
    printer(['V','V','X','X','X'])
    QNUM_Q = requests()
    printer(['V','V','V','X','X'])
    QNUM_D = qnum_docs()
    printer(['V','V','V','V','X'])
    TITLE_DOC = title_docid()
    printer(['V','V','V','V','V'])

In [19]:
"""Создаем папку с кортежами в виде: ./TuplesForNN/doc_id.pkl в которых лежат тайтлы"""
open_files()

In [13]:
"""Загрузка базовых словарей"""
on_start()

model______V
QNUM_Q_____V
QNUM_D_____V
TITLE_DOC__V


In [14]:
"""Загрузка сабмита"""
FinalRound()

399 queries handled
